### Standard Lib Imports

In [1]:
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [2]:
from src import config
from src.helper import *

### Load Data

In [3]:
prices = read_data('prices.csv')
product_details = read_data('product details.csv')

In [4]:
prices.head()

,product_id,sale_a1,sale_a2,sale_a3,sale_a4,purchase_a1,purchase_a2,purchase_a3,purchase_a4,valid_from,valid_to
0,270239865,18199,16799,12999,12199,10097,7851,3576,3290,2020-01-14T11:04:45.153+01:00,2020-01-14T12:45:30.042+01:00
1,337148188,242999,206399,197999,193999,177477,147779,141648,35553,2020-06-17T09:43:29.921+02:00,2020-06-18T09:16:31.401+02:00
2,341257331,70299,65399,57899,40499,54206,47594,44206,0,2020-02-28T10:50:17.647+01:00,2020-02-28T11:36:39.077+01:00
3,330479809,4099,3799,2299,1099,1287,1084,0,0,2020-02-28T11:39:46.102+01:00,2020-03-02T12:46:29.355+01:00
4,319791009,70799,61399,57899,32299,56158,48876,44067,22051,2020-01-27T08:15:27.795+01:00,2020-01-27T14:00:10.857+01:00


In [5]:
product_details = product_details.rename({'product_id*31': 'product_id'}, axis=1)
product_details.head(3)

,product_id,manufacturer,main_category
0,283509384,Apple,Handy
1,241663414,Tamron,Kameras
2,103621251,ASUSTeKCOMPUTER,Tablets


In [6]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655674 entries, 0 to 655673
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   product_id   655674 non-null  int64 
 1   sale_a1      655674 non-null  int64 
 2   sale_a2      655674 non-null  int64 
 3   sale_a3      655674 non-null  int64 
 4   sale_a4      655674 non-null  int64 
 5   purchase_a1  655674 non-null  int64 
 6   purchase_a2  655674 non-null  int64 
 7   purchase_a3  655674 non-null  int64 
 8   purchase_a4  655674 non-null  int64 
 9   valid_from   655674 non-null  object
 10  valid_to     655674 non-null  object
dtypes: int64(9), object(2)
memory usage: 55.0+ MB


In [7]:
prices['valid_from'] = pd.to_datetime(prices['valid_from'], utc=True)

In [8]:
# There is an issue while trying to convert 'valid_to' column because of some erroneuous values

There are some rows in the data which have some invalid/missing information. Let's drop these rows. 

In [9]:
problem_idx = []

for i, ts in enumerate(prices['valid_to']):
    try:
        pd.to_datetime(ts)
    except:
        problem_idx.append(i)

In [10]:
len(problem_idx)

622

In [11]:
prices = prices.drop(problem_idx).reset_index(drop=True)

In [12]:
# Now convert 'valid_to' to datetime column
prices['valid_to'] = pd.to_datetime(prices['valid_to'], utc=True)

In [13]:
prices.head()

,product_id,sale_a1,sale_a2,sale_a3,sale_a4,purchase_a1,purchase_a2,purchase_a3,purchase_a4,valid_from,valid_to
0,270239865,18199,16799,12999,12199,10097,7851,3576,3290,2020-01-14 10:04:45.153000+00:00,2020-01-14 11:45:30.042000+00:00
1,337148188,242999,206399,197999,193999,177477,147779,141648,35553,2020-06-17 07:43:29.921000+00:00,2020-06-18 07:16:31.401000+00:00
2,341257331,70299,65399,57899,40499,54206,47594,44206,0,2020-02-28 09:50:17.647000+00:00,2020-02-28 10:36:39.077000+00:00
3,330479809,4099,3799,2299,1099,1287,1084,0,0,2020-02-28 10:39:46.102000+00:00,2020-03-02 11:46:29.355000+00:00
4,319791009,70799,61399,57899,32299,56158,48876,44067,22051,2020-01-27 07:15:27.795000+00:00,2020-01-27 13:00:10.857000+00:00


In [14]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655052 entries, 0 to 655051
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   product_id   655052 non-null  int64              
 1   sale_a1      655052 non-null  int64              
 2   sale_a2      655052 non-null  int64              
 3   sale_a3      655052 non-null  int64              
 4   sale_a4      655052 non-null  int64              
 5   purchase_a1  655052 non-null  int64              
 6   purchase_a2  655052 non-null  int64              
 7   purchase_a3  655052 non-null  int64              
 8   purchase_a4  655052 non-null  int64              
 9   valid_from   655052 non-null  datetime64[ns, UTC]
 10  valid_to     655052 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), int64(9)
memory usage: 55.0 MB


In [15]:
prices['valid_period'] = prices['valid_to'] - prices['valid_from']

In [16]:
prices['margin_a1'] = prices['sale_a1'] - prices['purchase_a1']
prices['margin_a2'] = prices['sale_a2'] - prices['purchase_a2']
prices['margin_a3'] = prices['sale_a3'] - prices['purchase_a3']
prices['margin_a4'] = prices['sale_a4'] - prices['purchase_a4']

prices['pct_margin_a1'] = (prices['sale_a1'] - prices['purchase_a1'])/prices['purchase_a1']
prices['pct_margin_a2'] = (prices['sale_a2'] - prices['purchase_a2'])/prices['purchase_a2']
prices['pct_margin_a3'] = (prices['sale_a3'] - prices['purchase_a3'])/prices['purchase_a3']
prices['pct_margin_a4'] = (prices['sale_a4'] - prices['purchase_a4'])/prices['purchase_a4']

prices['avg_margin'] = round((prices['margin_a1'] + prices['margin_a2'] + prices['margin_a3'] + prices['margin_a4'])/4, 2)
prices['pct_avg_margin'] = round((prices['pct_margin_a1'] + prices['pct_margin_a2']\
                                  + prices['pct_margin_a3'] + prices['pct_margin_a4'])/4, 2)

In [17]:
prices = prices.replace(np.inf, 1)

In [18]:
prices.head()

,product_id,sale_a1,sale_a2,sale_a3,sale_a4,purchase_a1,purchase_a2,purchase_a3,purchase_a4,valid_from,...,margin_a1,margin_a2,margin_a3,margin_a4,pct_margin_a1,pct_margin_a2,pct_margin_a3,pct_margin_a4,avg_margin,pct_avg_margin
0,270239865,18199,16799,12999,12199,10097,7851,3576,3290,2020-01-14 10:04:45.153000+00:00,...,8102,8948,9423,8909,0.802417,1.139727,2.635067,2.707903,8845.50,1.82
1,337148188,242999,206399,197999,193999,177477,147779,141648,35553,2020-06-17 07:43:29.921000+00:00,...,65522,58620,56351,158446,0.369186,0.396673,0.397824,4.456614,84734.75,1.41
2,341257331,70299,65399,57899,40499,54206,47594,44206,0,2020-02-28 09:50:17.647000+00:00,...,16093,17805,13693,40499,0.296886,0.374102,0.309754,1.000000,22022.50,1.00
3,330479809,4099,3799,2299,1099,1287,1084,0,0,2020-02-28 10:39:46.102000+00:00,...,2812,2715,2299,1099,2.184926,2.504613,1.000000,1.000000,2231.25,1.00
4,319791009,70799,61399,57899,32299,56158,48876,44067,22051,2020-01-27 07:15:27.795000+00:00,...,14641,12523,13832,10248,0.260711,0.256220,0.313886,0.464741,12811.00,0.32


In [19]:
prices = prices.merge(product_details, on='product_id', how='left')

Let's look at the category wise margins across the variants

In [20]:
prices.groupby('main_category')[['margin_a1', 'margin_a2', 'margin_a3', 'margin_a4', 'avg_margin']]\
    .median().astype(int).sort_values(by='avg_margin', ascending=False)

,margin_a1,margin_a2,margin_a3,margin_a4,avg_margin
main_category,,,,,
Laptop,44190,42996,39185,47899,44350
Kameras,10025,10194,11070,12499,11261
Tablets,10556,11068,10034,10399,10475
Handy,9928,10309,9668,10704,10256
Wearables,8987,8759,8899,8717,9109
Apple,5976,5780,5399,5647,5801
Konsolen,5832,5417,5699,5299,5715
Audio & Hifi,5127,4937,4899,4799,5114
Electronic Accessories,3499,3499,3499,3499,3499


- Laptops, Cameras and Tablets are top 3 categories in terms of the **absolute** margins that rebuy is making.
- In general, the margin seems to be about the same across the variants. So there might not be any particular advantage to selling more of one variant over the other. This is good.

In [26]:
prices.groupby('main_category')[['pct_margin_a1', 'pct_margin_a2', 'pct_margin_a3', 'pct_margin_a4', 'pct_avg_margin']]\
    .mean().sort_values(by='pct_avg_margin', ascending=False)

,pct_margin_a1,pct_margin_a2,pct_margin_a3,pct_margin_a4,pct_avg_margin
main_category,,,,,
Laptop,0.888704,2.477779,1.035432,8.390053,2.835555
Tablets,1.835277,2.254766,2.132860,3.323808,1.716101
Handy,2.078759,2.405367,2.277796,2.953569,1.552646
Apple,1.555770,1.550246,1.589277,2.079335,1.269978
Kameras,0.925889,1.154258,1.193739,1.880692,1.163900
Konsolen,1.771256,2.071726,1.801844,1.374971,1.113045
Wearables,1.438525,1.574393,1.598013,1.621283,1.110355
Audio & Hifi,1.382160,1.447419,1.422900,1.257115,1.055772
Electronic Accessories,1.000000,1.000000,1.000000,1.000000,1.000000


- % Margin for A4 of Laptop is extremely high. We would need to dig deeper and understand if this is legitimate. But if it is, then it's a good sign!